# Hands-on 4: Abstract Gate

In this notebook, you will learn to create and use abstract gates.

Abstract Gates are usefull to:
- ease de developpement of a circuit by black boxing some part of the algorithm
- black boxing a routine call which is known to be available on a specific hardware

We will provide a matrix generator in order to be able to populate the matrices in the circuit model, thus allowing us to later simulate these circuits.

## Constant abstract gates

We will first create a CNOT with an abstract gate.

The first step is to import AbstractGate from qat.lang.AQASM:

In [ ]:
from XXX.XXX.XXX import XXX

We then need to create our gate by saving the result of the function AbstractGate():
- Choose a name for your CNOT gate

Next complete the parameters of the AbstractGate function:
- Choose the name that will be displayed when printing your circuit
- The CNOT gate doesn't need any parameters so keep [] empty
- Fix the arity of the CNOT


In [ ]:
# CNOT takes no parameter and is of arity 2
XXX = AbstractGate("XXX", [], arity=XXX)

Once you have defined your abstract gate you can apply it. 
The main difference is: your abstract gate is a function so you need to call it with empty parenthesis (like a function).

For example we will use the example of the EPR pair. The single thing you need to complete is the name of your abstract gate:

In [ ]:
from qat.lang.AQASM import Program,H
prog = Program()
qbits = prog.qalloc(2)
prog.apply(H, qbits[0])
prog.apply(XXX(), qbits)
circuit = prog.to_circ()
%qatdisplay circuit

It is not possible to simulate this circuit since the matrix is not known.

Execute the following cell to see the error when you try to simulate such circuit:

In [ ]:
from qat.qpus import LinAlg
linalgqpu = LinAlg()

job = circuit.to_job()

try:
    for sample in linalgqpu.submit(job):
        print("State %s amplitude %s" % (sample.state, sample.amplitude))
        
except Exception as e:
    print(type(e), e)

If we want to simulate we need to specify the matrix of our gate.

The usual way to define the CNOT's matrix is :
$$\begin{bmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$$

Complete the following cell with the matrix' definition:

In [ ]:
import numpy as np

def CNOT_generator():
    return np.array([[XXX, XXX, XXX, XXX],
                     [XXX, XXX, XXX, XXX],
                     [XXX, XXX, XXX, XXX],
                     [XXX, XXX, XXX, XXX]])

XXX.set_matrix_generator(CNOT_generator)

Let's create the program as previously:

In [ ]:
from qat.lang.AQASM import Program,H
prog = Program()
qbits = prog.qalloc(2)
prog.apply(H, qbits[0])
prog.apply(XXX(), qbits)
circuit = prog.to_circ()

Since we have now our circuit with our gate properly definied we can simulate it:

In [ ]:
from qat.qpus import LinAlg
linalgqpu = LinAlg()

job = circuit.to_job()
try:
    for sample in linalgqpu.submit(job):
        print("State %s amplitude %s" % (sample.state, sample.amplitude))
        
except Exception as e:
    print(type(e), e)

You should obtain :
- State |00> amplitude (0.7071067811865475+0j)
- State |11> amplitude (0.7071067811865475+0j)

## Parametrized abstract gates

Let's now have a look at how to create a parametrized abstract gate.

We will define some kind of parametrized CNOT for example:

$$\begin{bmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & e^{-\frac{\theta i }{2}} \\ 0 & 0 & e^{\frac{\theta i }{2}} & 0 \end{bmatrix}$$

Define this matrix in the following cell.

Notes : 
- use 1j for i.
- use np.exp for the exponential
- define the name of your variable for $\theta$

In [ ]:
import numpy as np
# Let us define a function that given a value for theta returns a matrix corresponding to CX(theta)
# Our CX(theta) will simply be some kind of parametrized CNOT
def CX_generator(theta):
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 0, XXX],
                     [0, 0, XXX, 0]])

We can know define our gate:
- Find a name for your gate
- Define the name when diplaying the circuit
- Define the type of the parameter of our gate
- Define the arity of our gate
- Attribute our generator

In [ ]:
# Equivalently, we could have defined the gate directly as follows:
XXX = AbstractGate("XXX", [XXX], arity=XXX, matrix_generator=XXX)

Let's create a simple circuit with an Hadamard and our CX gate with an angle of 0.33:

In [ ]:
prog = Program()

qbits = prog.qalloc(2)

prog.apply(H, qbits[0])
prog.apply(XXX(XXX), qbits)

circuit = prog.to_circ()
%qatdisplay circuit

We can directly simlulate our circuit since we have defined entirely our gate:

In [ ]:
from qat.qpus import LinAlg
linalgqpu = LinAlg()

job = circuit.to_job()

try:
    for sample in linalgqpu.submit(job):
        print("State %s amplitude %s" % (sample.state, sample.amplitude))
        
except Exception as e:
    print(type(e), e)

The circuit with an angle of 0.33 should give you:
- State |11> amplitude (0.6975031081522839+0.1161439370690664j)
- State |00> amplitude (0.7071067811865475+0j)

## Much more tricky 

In this last section you can see a more complex definition for abstract gates:
- using two angles
- combining matrices to define yours


In [ ]:
from qat.lang.AQASM import AbstractGate,Program,H
from qat.qpus import LinAlg
import numpy as np

PX = AbstractGate("PX", [float, float], arity=1)

def PX_generator(phi, theta):
    _I = np.eye(2, dtype=np.complex128)
    _X = np.array([[0,1],[1,0]], dtype=np.complex128)
    _Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
    return np.cos(theta/2) * _I - 1j * np.sin(theta/2) *(np.cos(phi) * _X + np.sin(phi) * _Y)

PX.set_matrix_generator(PX_generator)

prog = Program()
qbits = prog.qalloc(1)
prog.apply(PX(1.33, 0.4), qbits[0])
circuit = prog.to_circ()
%qatdisplay circuit

linalgqpu = LinAlg()
job = circuit.to_job()
for sample in linalgqpu.submit(job):
    print("State %s amplitude %s" % (sample.state, sample.amplitude))

As you have seen, abstract gates can help you during development phase to put in place the big frame or black boxing specific available gates.